In [1]:
##
## Import various modules 
##
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib tk
import sys
import os
import re
from scipy import io
import pickle
import copy
from iris_lmsalpy import extract_irisL2data
sys.path.insert(0,'/Users/gskerr1/Documents/Research/Python_Programs/iris_programs')
#import importlib
import IRIS_SG_deconvolve
#importlib.reload(IRIS_SG_deconvolve)

In [2]:
###################
###################
##
## Graham S. Kerr
## NASA/GSFC & CUA
## July 2020
## 
## NAME:   IRIS_SG_Deconvolve.py
##
## PURPOSE: Deconvolves IRIS SG data using the PSFs from Courrier et al 2018.
##          
## NOTES: Calls IRIS_SG_deconvolve.py
##
##        There are probably more clever ways to code this in -- i'm fairly new to python. 
##        
##        Tested on Mg II data from 2015-March-11
##
##        Compared to the IDL version, numbers all seem to match
##
###################
###################

In [3]:
## 
## Some prelim set up
## 

# Number of Richardson-Lucy iterations to perform (this will be an input eventually)
iterations = 10

# Which raster image to focus on 
rind = 0

# Which window are we looking at? 
psfname = 'sg_psf_2796'

# To deconvolve by division set to True
fft_div = False

In [4]:
##
## The SG filenames
## 

dir1 = './'
raster_filename = dir1+'iris_l2_20150311_151947_3860107071_raster_t000_r00170.fits'

In [6]:
## 
## Load the PSFs
##

psf_filename = dir1+'IRIS_SG_PSFs.dictionary'
with open(dir1+psf_filename, 'rb') as config_dictionary_file:
    psf_all = pickle.load(config_dictionary_file)

if psfname == 'sg_psf_2796':
    psf = psf_all['sg_psf_2796']
elif psfname == 'sg_psf_2814':
    psf = psf_all['sg_psf_2814']
elif psfname == 'sg_psf_1336':
    psf = psf_all['sg_psf_1336'] 
elif psfname == 'sg_psf_1394':
    psf = psf_all['sg_psf_1394']  
else: 
    psf = psf_all['sg_psf_2796']
    print('Defaulting to sg_psf_2796')

In [7]:
##
## Show the obs obserview
##

hdr = extract_irisL2data.only_header(raster_filename)
hdr['OBS_DESC']

'Large coarse 4-step raster 6x120 4s  C II   Si IV   Mg II w Deep x 4'

In [8]:
##
## Show the line info and winids
##

lines=extract_irisL2data.show_lines(raster_filename)


Extracting information from file ./iris_l2_20150311_151947_3860107071_raster_t000_r00170.fits... 

Available data with size Y x X x Wavelength are stored in windows labeled as:

--------------------------------------------------------------------
Index --- Window label --- Y x X x WL --- Spectral range [AA] (band)
--------------------------------------------------------------------
  0 	 C II 1336    	   388x4x170 	   1332.75 - 1337.14  (FUV)
  1 	 1343         	   388x4x73 	   1342.33 - 1344.20  (FUV)
  2 	 Fe XII 1349  	   388x4x121 	   1347.73 - 1350.85  (FUV)
  3 	 O I 1356     	   388x4x138 	   1352.28 - 1355.83  (FUV)
  4 	 Si IV 1403   	   388x4x284 	   1398.68 - 1405.88  (FUV)
  5 	 2832         	   388x4x56 	   2831.39 - 2834.19  (NUV)
  6 	 2826         	   388x4x72 	   2824.87 - 2828.49  (NUV)
  7 	 2814         	   388x4x72 	   2812.70 - 2816.32  (NUV)
  8 	 Mg II k 2796 	   388x4x312 	   2790.71 - 2806.54  (NUV)
------------------------------------------------------------

In [9]:
##
## Create a raster object with certain windows
##

iris_sg = extract_irisL2data.load(raster_filename, window_info = lines[[7,8]])

Creating temporary file...  /var/folders/xb/7zrk8dcn2kj26gcgxfjmyg9cv_zfzh/T/tmp8ek_9k3y
Creating temporary file...  /var/folders/xb/7zrk8dcn2kj26gcgxfjmyg9cv_zfzh/T/tmpxge4kfje


In [10]:
##
## Load into local memory so we can modify the data
##

iris_sg.flush()

Removing temporary file... /var/folders/xb/7zrk8dcn2kj26gcgxfjmyg9cv_zfzh/T/tmp8ek_9k3y
Removing temporary file... /var/folders/xb/7zrk8dcn2kj26gcgxfjmyg9cv_zfzh/T/tmpxge4kfje


In [11]:
##
## Deconvolution of PSF
##

data_in = iris_sg.raster['Mg II k 2796'].data[:,rind,:]
data_decon = IRIS_SG_deconvolve.IRIS_SG_deconvolve(data_in,psf,iterations=iterations, fft_div=fft_div)

/Users/gskerr1/Documents/Research/Python_Programs/iris_programs/IRIS_SG_deconvolve.py:49: RuntimeWarning: invalid value encountered in true_divide
  step1 = data_in_zr/(FFT_conv_1D(dcvim,psf,rev_psf=False,div=False))


In [12]:
plt.imshow(data_decon**0.3,origin='lower')